## For TPC-DS dataset with the following query tempalte:
SELECT ss_store_sk,ss_coupon_amt, AF(ss_sales_price)

FROM store_sales

WHERE ss_sold_date_sk BETWEEN low AND high


GROUP BY ss_store_sk,ss_quantity

In [1]:
from dbestclient.executor.executor import SqlExecutor

## Parameter Initialization

In [2]:
sqlExecutor = SqlExecutor()
sqlExecutor.execute("set csv_split_char='|'")
sqlExecutor.execute("set encoder='embedding'")
sqlExecutor.execute("set n_epoch=20")
sqlExecutor.execute("set n_mdn_layer_node_reg=50")          
sqlExecutor.execute("set n_mdn_layer_node_density=60")      
sqlExecutor.execute("set n_hidden_layer=1")                       
sqlExecutor.execute("set n_gaussians_reg=8")                
sqlExecutor.execute("set n_gaussians_density=12")            
sqlExecutor.execute("set n_embedding_dim=20")
sqlExecutor.execute(
    "set table_header="
    + "'ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|"
    + "ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|"
    + "ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|"
    + "ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|"
    + "ss_net_paid_inc_tax|ss_net_profit|none'"
)

Local mode is on, as no slaves are provided.
start loading pre-existing models.
Loaded 1 models. time cost  0.004136 s
OK, csv_split_char is updated.
OK, encoder is updated.
OK, n_epoch is updated.
OK, table_header is updated.


## Model Creation

In [1]:
# sqlExecutor.execute("drop table template3")  # drop the model
sqlExecutor.execute(
    "create table template3(ss_sales_price real, ss_sold_date_sk real)  from '/home/quincy/Documents/workspace/data/tpcds/1t/ss_1m.csv' GROUP BY ss_store_sk,ss_quantity method uniform size 0.0003472"
)

NameError: name 'sqlExecutor' is not defined

## Query Serving

In [ ]:
predictions = sqlExecutor.execute(
            "select ss_store_sk, avg(ss_sales_price)  from template3 where   2451119  <=ss_sold_date_sk<= 2451483  group by ss_store_sk,ss_quantity"
        )

# sqlExecutor.execute("drop table template3")

## HIVE query
select ss_store_sk, count(ss_sales_price),sum(ss_sales_price),avg(ss_sales_price)  from store_sales_1t where  ss_sold_date_sk BETWEEN 2451119  AND 2451483  group by ss_store_sk,ss_quantity